In [1]:
import os 
os.chdir('../../')
print("Current working directory is now: ", os.getcwd())

import pandas as pd 
import numpy as np
import csv
from sklearn.linear_model import LogisticRegression

import KY_utils.KY_interpretable_functions as interpret
import KY_utils.KY_RiskSLIM as slim
import KY_utils.KY_stumps as stumps

Current working directory is now:  C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\kentucky\ky_fl_combined\KY Model


C:\Users\binha\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning:

The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).



### EBM & CART

In [2]:
KY_data = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/KY_cleaned")
FL_data = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/FL_cleaned")

## split x and y 
KY_X = KY_data.loc[:,:'current_violence20']
KY_Y = KY_data['property_two_year'].values
FL_X = FL_data.loc[:,:'current_violence20']
FL_Y = FL_data['property_two_year'].values

In [3]:
#### CART
depth = [8,9,10]
cart_summary = interpret.CART(KY_X, KY_Y, FL_X, FL_Y, depth, seed=816)

### EBM
estimators = [40]
depth = [2]
learning_rate = [0.7]
ebm_summary = interpret.EBM(KY_X, KY_Y, FL_X, FL_Y, learning_rate, depth, estimators, seed=816)

In [4]:
cart_summary['FL_score'],  ebm_summary['FL_score']

(0.6210744398940014, 0.6728033405605076)

### Lasso Stumps

In [7]:
KY_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/KY_stumps")
FL_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/FL_stumps")

## split x and y 
KY_X = KY_stumps.loc[:,:'current_violence201']
KY_Y = KY_stumps['property_two_year'].values
FL_X = FL_stumps.loc[:,:'current_violence201']
FL_Y = FL_stumps['property_two_year'].values

## columns 
cols = KY_X.columns

In [8]:
single_stump_model = stumps.stump_model(KY_X, KY_Y, FL_X, FL_Y, 0.004, cols, 816)
single_stump_model['features']

['sex1',
 'age_at_current_charge18.0',
 'age_at_current_charge19.0',
 'age_at_current_charge20.0',
 'age_at_current_charge22.0',
 'age_at_current_charge26.0',
 'age_at_current_charge27.0',
 'age_at_current_charge35.0',
 'age_at_current_charge37.0',
 'age_at_current_charge38.0',
 'age_at_current_charge39.0',
 'age_at_current_charge41.0',
 'age_at_current_charge44.0',
 'age_at_current_charge45.0',
 'age_at_current_charge48.0',
 'age_at_current_charge50.0',
 'age_at_current_charge70.0',
 'p_arrest1',
 'p_arrest2',
 'p_arrest3',
 'p_arrest4',
 'p_arrest6',
 'p_arrest7',
 'p_arrest8',
 'p_charges1',
 'p_charges5',
 'p_charges6',
 'p_violence1',
 'p_felony1',
 'p_felony2',
 'p_felony3',
 'p_misdemeanor1',
 'p_property1',
 'p_property2',
 'p_drug1',
 'p_stealing1',
 'p_fta_two_year1',
 'p_pending_charge1',
 'p_probation1',
 'p_probation2',
 'p_incarceration1',
 'five_year1']

In [9]:
c_grid={'C': [0.001, 0.002, 0.003, 0.004]}
stumps_summary = stumps.stump_cv(KY_X, KY_Y, FL_X, FL_Y, cols, c_grid, seed=816)

In [10]:
stumps_summary['best_params'], stumps_summary['FL_score'], stumps_summary['auc_diffs']

({'C': 0.004}, 0.6658732835461334, 0.0011422931438835882)

### RiskSLIM

In [11]:
KY_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/KY_stumps")
FL_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/FL_stumps")

## split x and y 
KY_X = KY_stumps.loc[:,:'current_violence201']
KY_Y = KY_stumps['property_two_year'].values
FL_X = FL_stumps.loc[:,:'current_violence201']
FL_Y = FL_stumps['property_two_year'].values

## columns 
cols = KY_X.columns

In [17]:
## train on best param chosen by Lasso Stumps from above
single_stump_model = stumps.stump_model(KY_X, KY_Y, FL_X, FL_Y, 
                                        0.0005, 
                                        cols, 
                                        816)
selected_features = single_stump_model['features']
len(selected_features)

15

In [18]:
### Subset features
sub_FL_X = FL_stumps.loc[:, selected_features]
sub_KY_X = KY_stumps.loc[:, selected_features]
sub_KY_X.insert(0, '(Intercept)', 1)

In [19]:
riskslim_summary = slim.risk_cv(sub_KY_X, KY_Y, sub_FL_X, FL_Y, 
                                y_label = 'property_two_year', 
                                max_coef = 5, 
                                max_coef_number = 5, 
                                max_runtime=1000,
                                max_offset=100,
                                c=1e-3, 
                                seed=816)

setting c0 = 0.0 to ensure that intercept is not penalized
11/12/19 @ 02:49 PM | 203 rows in lookup table
11/12/19 @ 02:49 PM | ------------------------------------------------------------
11/12/19 @ 02:49 PM | runnning initialization procedure
11/12/19 @ 02:49 PM | ------------------------------------------------------------
11/12/19 @ 02:49 PM | CPA produced 2 cuts
11/12/19 @ 02:49 PM | running naive rounding on 8 solutions
11/12/19 @ 02:49 PM | best objective value: 0.2131
11/12/19 @ 02:49 PM | rounding produced 5 integer solutions
11/12/19 @ 02:49 PM | best objective value is 0.2267
11/12/19 @ 02:49 PM | running sequential rounding on 8 solutions
11/12/19 @ 02:49 PM | best objective value: 0.2131
11/12/19 @ 02:49 PM | sequential rounding produced 4 integer solutions
11/12/19 @ 02:49 PM | best objective value: 0.2155
11/12/19 @ 02:49 PM | polishing 9 solutions
11/12/19 @ 02:49 PM | best objective value: 0.2155
11/12/19 @ 02:49 PM | polishing produced 5 integer solutions
11/12/19 @ 0

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/12/19 @ 02:49 PM | adding 235 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.2137.
Tried aggregator 1 time.
Reduced MIP has 32 rows, 34 columns, and 93 nonzeros.
Reduced MIP has 15 binaries, 17 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

setting c0 = 0.0 to ensure that intercept is not penalized
11/12/19 @ 02:50 PM | 203 rows in lookup table
11/12/19 @ 02:50 PM | ------------------------------------------------------------
11/12/19 @ 02:50 PM | runnning initialization procedure
11/12/19 @ 02:50 PM | ------------------------------------------------------------
11/12/19 @ 02:50 PM | CPA produced 2 cuts
11/12/19 @ 02:50 PM | running naive rounding on 7 solutions
11/12/19 @ 02:50 PM | best objective value: 0.2260
11/12/19 @ 02:50 PM | rounding produced 5 integer solutions
11/12/19 @ 02:50 PM | best objective value is 0.2369
11/12/19 @ 02:50 PM | running sequential rounding on 7 solutions
11/12/19 @ 02:50 PM | best objective value: 0.2260
11/12/19 @ 02:50 PM | sequential rounding produced 4 integer solutions
11/12/19 @ 02:50 PM | best objective value: 0.2184
11/12/19 @ 02:50 PM | polishing 9 solutions
11/12/19 @ 02:50 PM | best objective value: 0.2184
11/12/19 @ 02:50 PM | polishing produced 5 integer solutions
11/12/19 @ 0

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/12/19 @ 02:50 PM | adding 235 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.2168.
Tried aggregator 1 time.
Reduced MIP has 32 rows, 34 columns, and 93 nonzeros.
Reduced MIP has 15 binaries, 17 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

11/12/19 @ 02:51 PM | ------------------------------------------------------------
11/12/19 @ 02:51 PM | 203 rows in lookup table
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
CPXPARAM_Parallel                                1
CPXPARAM_RandomSeed                              0
CPXPARAM_TimeLimit                               2000
CPXPARAM_MIP_Tolerances_LowerCutoff              0.20485153508535231
CPXPARAM_MIP_Tolerances_UpperCutoff              0.22196628772717072


Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/12/19 @ 02:51 PM | adding 233 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.2220.
Tried aggregator 1 time.
Reduced MIP has 32 rows, 34 columns, and 93 nonzeros.
Reduced MIP has 15 binaries, 17 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/12/19 @ 02:52 PM | adding 242 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.2242.
Tried aggregator 1 time.
Reduced MIP has 32 rows, 34 columns, and 93 nonzeros.
Reduced MIP has 15 binaries, 17 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/12/19 @ 02:54 PM | adding 240 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.2235.
Tried aggregator 1 time.
Reduced MIP has 32 rows, 34 columns, and 93 nonzeros.
Reduced MIP has 15 binaries, 17 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

setting c0 = 0.0 to ensure that intercept is not penalized
11/12/19 @ 02:55 PM | 203 rows in lookup table
11/12/19 @ 02:55 PM | ------------------------------------------------------------
11/12/19 @ 02:55 PM | runnning initialization procedure
11/12/19 @ 02:55 PM | ------------------------------------------------------------
11/12/19 @ 02:55 PM | CPA produced 2 cuts
11/12/19 @ 02:55 PM | running naive rounding on 6 solutions
11/12/19 @ 02:55 PM | best objective value: 0.5646
11/12/19 @ 02:55 PM | rounding produced 5 integer solutions
11/12/19 @ 02:55 PM | best objective value is 0.5319
11/12/19 @ 02:55 PM | running sequential rounding on 6 solutions
11/12/19 @ 02:55 PM | best objective value: 0.5646
11/12/19 @ 02:55 PM | sequential rounding produced 3 integer solutions
11/12/19 @ 02:55 PM | best objective value: 0.3639
11/12/19 @ 02:55 PM | polishing 8 solutions
11/12/19 @ 02:55 PM | best objective value: 0.3639
11/12/19 @ 02:55 PM | polishing produced 4 integer solutions
11/12/19 @ 0

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/12/19 @ 02:55 PM | adding 228 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.2233.
Tried aggregator 1 time.
Reduced MIP has 32 rows, 34 columns, and 93 nonzeros.
Reduced MIP has 15 binaries, 17 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

In [20]:
riskslim_summary['FL_score'], np.mean(riskslim_summary['KY_validation_score'])

(0.5994298562595358, 0.7457583528465822)

In [22]:
#### save results
summary_property2_ky_inter_model = {"CART": cart_summary,
                                   "EBM": ebm_summary,
                                   "Stumps": stumps_summary,
                                   "RiskSLIM": riskslim_summary}

In [23]:
path = "C:\\Users\\binha\\Documents\\Duke\\Cynthia Research\\KY-analysis-mytrials\\KY Recidivism\\KY Results\\KY_FL\\KY Model\\interpretable\\Two Year\\"
results = [["Property",             
            round(cart_summary['FL_score'],3), 
            round(ebm_summary['FL_score'], 3), 
            round(stumps_summary['FL_score'],3), 
            round(riskslim_summary['FL_score'], 3)]]
with open(path + 'Two Year KY_inter_model.csv', 'a') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(results)